In [1]:
#!/usr/bin/env python
# coding=utf-8

__author__ = "Xinpeng.Chen"

import os
import sys
import time
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import random

import h5py
import ipdb

import tensorflow as tf


# ------------------------------------------------------------------------------------------------------
# Initialization class
#  1. Pooling the visual features into a single dense feature
#  2. Then, build sentence LSTM, word LSTM
# ------------------------------------------------------------------------------------------------------
class RegionPooling_HierarchicalRNN():
    def __init__(self, n_words,
                       batch_size,
                       num_boxes,
                       feats_dim,
                       project_dim,
                       sentRNN_lstm_dim,
                       sentRNN_FC_dim,
                       wordRNN_lstm_dim,
                       S_max,
                       N_max,
                       word_embed_dim,
                       bias_init_vector=None):

        self.n_words = n_words
        self.batch_size = batch_size
        self.num_boxes = num_boxes # 50
        self.feats_dim = feats_dim # 4096
        self.project_dim = project_dim # 1024
        self.S_max = S_max # 6
        self.N_max = N_max # 50
        self.word_embed_dim = word_embed_dim # 1024

        self.sentRNN_lstm_dim = sentRNN_lstm_dim # 512 hidden size
        self.sentRNN_FC_dim = sentRNN_FC_dim # 1024 in fully connected layer
        self.wordRNN_lstm_dim = wordRNN_lstm_dim # 512 hidden size
	


        # word embedding, parameters of embedding
        # embedding shape: n_words x wordRNN_lstm_dim
        with tf.device('/cpu:0'):
            self.Wemb = tf.Variable(tf.random_uniform([n_words, word_embed_dim], -0.1, 0.1), name='Wemb')
        print("Wemb output>>>>>>>>>>>>",self.Wemb,tf.shape(self.Wemb))
        #self.bemb = tf.Variable(tf.zeros([word_embed_dim]), name='bemb')

        # regionPooling_W shape: 4096 x 1024
        # regionPooling_b shape: 1024
        self.regionPooling_W = tf.Variable(tf.random_uniform([feats_dim, project_dim], -0.1, 0.1), name='regionPooling_W')
        self.regionPooling_b = tf.Variable(tf.zeros([project_dim]), name='regionPooling_b')

        # sentence LSTM
        self.sent_LSTM = tf.nn.rnn_cell.BasicLSTMCell(sentRNN_lstm_dim, state_is_tuple=True)

        # logistic classifier
        self.logistic_Theta_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, 2], -0.1, 0.1), name='logistic_Theta_W')
        self.logistic_Theta_b = tf.Variable(tf.zeros(2), name='logistic_Theta_b')

        # fc1_W: 512 x 1024, fc1_b: 1024
        # fc2_W: 1024 x 1024, fc2_b: 1024
        self.fc1_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, sentRNN_FC_dim], -0.1, 0.1), name='fc1_W')
        self.fc1_b = tf.Variable(tf.zeros(sentRNN_FC_dim), name='fc1_b')
        self.fc2_W = tf.Variable(tf.random_uniform([sentRNN_FC_dim, 1024], -0.1, 0.1), name='fc2_W')
        self.fc2_b = tf.Variable(tf.zeros(1024), name='fc2_b')
        def get_a_cell(lstm_size):
          lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
          return lstm
        # word LSTM
        self.word_LSTM = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)
        #self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([self.word_LSTM] * 2, state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([get_a_cell(wordRNN_lstm_dim) for i in range(2)], state_is_tuple=True)
        #self.word_LSTM2 = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)


        self.embed_word_W = tf.Variable(tf.random_uniform([wordRNN_lstm_dim, n_words], -0.1,0.1), name='embed_word_W')
        if bias_init_vector is not None:
            self.embed_word_b = tf.Variable(bias_init_vector.astype(np.float32), name='embed_word_b')
        else:
            self.embed_word_b = tf.Variable(tf.zeros([n_words]), name='embed_word_b')

    def build_model(self):
        # receive the feats in the current image
        # it's shape is 10 x 50 x 4096
        # tmp_feats: 500 x 4096
        feats = tf.placeholder(tf.float32, [self.batch_size, self.num_boxes, self.feats_dim])
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 500 x 4096 * 4096 x 1024 --> 500 x 1024
        # project_vec: 10 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [self.batch_size, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # receive the [continue:0, stop:1] lists
        # example: [0, 0, 0, 0, 1, 1], it means this paragraph has five sentences
        num_distribution = tf.placeholder(tf.int32, [self.batch_size, self.S_max])

        # receive the ground truth words, which has been changed to idx use word2idx function
        captions = tf.placeholder(tf.int32, [self.batch_size, self.S_max, self.N_max+1])
        #print("Captions:>>>>>>>>>>>>>>>>>>>>>>>",captions)
        captions_masks = tf.placeholder(tf.float32, [self.batch_size, self.S_max, self.N_max+1])

        # ---------------------------------------------------------------------------------------------------------------------
        # The method which initialize the state, is refered from below sites:
        # 1. http://stackoverflow.com/questions/38241410/tensorflow-remember-lstm-state-for-next-batch-stateful-lstm/38417699
        # 2. https://www.tensorflow.org/api_docs/python/rnn_cell/classes_storing_split_rnncell_state#LSTMStateTuple
        # 3. https://medium.com/@erikhallstrm/using-the-tensorflow-lstm-api-3-7-5f2b97ca6b73#.u4w9z6h0h
        # ---------------------------------------------------------------------------------------------------------------------
        sent_state = self.sent_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state = self.word_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state1 = self.word_LSTM1.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state2 = self.word_LSTM2.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #sent_state = tf.zeros([self.batch_size, self.sent_LSTM1.state_size])
        #word_state1 = tf.zeros([self.batch_size, self.word_LSTM1.state_size])
        #word_state2 = tf.zeros([self.batch_size, self.word_LSTM2.state_size])

        probs = []
        loss = 0.0
        loss_sent = 0.0
        loss_word = 0.0
        lambda_sent = 5.0
        lambda_word = 1.0

        print('Start build model:')
        #----------------------------------------------------------------------------------------------
        # Hierarchical RNN: sentence RNN and words RNN
        # The word RNN has the max number, N_max = 50, the number in the papar is 50
        #----------------------------------------------------------------------------------------------
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            with tf.variable_scope('sent_LSTM',reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            # sent_state is a tuple, sent_state = (c, h)
            # 'c': shape=(1, 512) dtype=float32, 'h': shape=(1, 512) dtype=float32
            # The loss here, I refer from the web which is very helpful for me:
            # 1. http://stackoverflow.com/questions/34240703/difference-between-tensorflow-tf-nn-softmax-and-tf-nn-softmax-cross-entropy-with
            # 2. http://stackoverflow.com/questions/35277898/tensorflow-for-binary-classification
            # 3. http://stackoverflow.com/questions/35226198/is-this-one-hot-encoding-in-tensorflow-fast-or-flawed-for-any-reason
            # 4. http://stackoverflow.com/questions/35198528/reshape-y-train-for-binary-text-classification-in-tensorflow
            sentRNN_logistic_mu = tf.nn.xw_plus_b( sent_output, self.logistic_Theta_W, self.logistic_Theta_b )
            sentRNN_label = tf.stack([ 1 - num_distribution[:, i], num_distribution[:, i] ])
            sentRNN_label = tf.transpose(sentRNN_label)
            sentRNN_loss = tf.nn.softmax_cross_entropy_with_logits(logits=sentRNN_logistic_mu, labels=sentRNN_label)
            sentRNN_loss = tf.reduce_sum(sentRNN_loss)/self.batch_size
            loss += sentRNN_loss * lambda_sent
            loss_sent += sentRNN_loss

            # the begining input of word_LSTM is topic vector, and DON'T compute the loss
            # This is follow the paper: Show and Tell
            #word_state = self.word_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
            #with tf.variable_scope('word_LSTM'):
            #    word_output, word_state = self.word_LSTM(sent_topic_vec)
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, captions[:, i, j])

                with tf.variable_scope('word_LSTM',reuse=tf.AUTO_REUSE):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                # How to make one-hot encoder, I refer from this excellent web:
                # http://stackoverflow.com/questions/33681517/tensorflow-one-hot-encoder
                labels = tf.reshape(captions[:, i, j+1], [-1, 1])
                #print("Labels and its shape +++++++++++++++",labels,tf.shape(labels))
                indices = tf.reshape(tf.range(0, self.batch_size, 1), [-1, 1])
                #print("Indices and its shape +++++++++++++++",indices,tf.shape(indices))
                concated = tf.concat([indices, labels],1)
                #print("Concated+++++++++++++++++++",concated)
                print("Success")
                onehot_labels = tf.sparse_to_dense(concated, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)

                # At each timestep the hidden state of the last LSTM layer is used to predict a distribution
                # over the words in the vocbulary
                logit_words = tf.nn.xw_plus_b(word_output[:], self.embed_word_W, self.embed_word_b)
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=onehot_labels)
                cross_entropy = cross_entropy * captions_masks[:, i, j]
                loss_wordRNN = tf.reduce_sum(cross_entropy) / self.batch_size
                loss += loss_wordRNN * lambda_word
                loss_word += loss_wordRNN

        return feats, num_distribution, captions, captions_masks, loss, loss_sent, loss_word

    def generate_model(self):
        # feats: 1 x 50 x 4096
        feats = tf.placeholder(tf.float32, [1, self.num_boxes, self.feats_dim])
        # tmp_feats: 50 x 4096
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 50 x 4096 * 4096 x 1024 + 1024 --> 50 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [1, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # initialize the sent_LSTM state
        sent_state = self.sent_LSTM.zero_state(batch_size=1, dtype=tf.float32)

        # save the generated paragraph to list, here I named generated_sents
        generated_paragraph = []

        # pred
        pred_re = []

        # T_stop: run the sentence RNN forward until the stopping probability p_i (STOP) exceeds a threshold T_stop
        T_stop = tf.constant(0.5)

        # Start build the generation model
        print('Start build the generation model: ')

        # sentence RNN
        #word_state = self.word_LSTM.zero_state(batch_size=1, dtype=tf.float32)
        #with tf.variable_scope('word_LSTM'):
        #    word_output, word_state = self.word_LSTM(sent_topic_vec, word_state)
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            # sent_state:
            # LSTMStateTuple(c=<tf.Tensor 'sent_LSTM/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
            #                h=<tf.Tensor 'sent_LSTM/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>)
            with tf.variable_scope('sent_LSTM',reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            # self.fc1_W: 512 x 1024, self.fc1_b: 1024
            # hidden1: 1 x 1024
            # sent_topic_vec: 1 x 1024
            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            sentRNN_logistic_mu = tf.nn.xw_plus_b(sent_output, self.logistic_Theta_W, self.logistic_Theta_b)
            pred = tf.nn.softmax(sentRNN_logistic_mu)
            pred_re.append(pred)

            # save the generated sentence to list, named generated_sent
            generated_sent = []

            # initialize the word LSTM state
            #word_state = self.word_LSTM.zero_state(batch_size=1, dtype=tf.float32)
            #with tf.variable_scope('word_LSTM'):
            #    word_output, word_state = self.word_LSTM(sent_topic_vec, word_state)
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            # word RNN, unrolled to N_max time steps
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                if j == 0:
                    with tf.device('/cpu:0'):
                        # get word embedding of BOS (index = 0)
                        current_embed = tf.nn.embedding_lookup(self.Wemb, tf.zeros([1], dtype=tf.int64))

                with tf.variable_scope('word_LSTM',reuse=tf.AUTO_REUSE):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                # word_state:
                # (
                #     LSTMStateTuple(c=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell0/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
                #                    h=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell0/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>),
                #     LSTMStateTuple(c=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell1/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
                #                    h=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>)
                # )
                logit_words = tf.nn.xw_plus_b(word_output, self.embed_word_W, self.embed_word_b)
                max_prob_index = tf.argmax(logit_words, 1)[0]
                generated_sent.append(max_prob_index)

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, max_prob_index)
                    current_embed = tf.expand_dims(current_embed, 0)

            generated_paragraph.append(generated_sent)

        return feats, generated_paragraph, pred_re, sent_topic_vec


# -----------------------------------------------------------------------------------------------------
# Preparing Functions
# -----------------------------------------------------------------------------------------------------
def preProBuildWordVocab(sentence_iterator, word_count_threshold=5):
    # borrowed this function from NeuralTalk
    print('preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, ))

    word_counts = {}
    nsents = 0

    for sent in sentence_iterator:
        nsents += 1
        tmp_sent = sent.lower().split(' ')
        if '' in tmp_sent:
            tmp_sent.remove('')

        for w in tmp_sent:
           word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print('filtered words from %d to %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '<bos>'
    ixtoword[1] = '<eos>'
    ixtoword[2] = '<pad>'
    ixtoword[3] = '<unk>'

    wordtoix = {}
    wordtoix['<bos>'] = 0
    wordtoix['<eos>'] = 1
    wordtoix['<pad>'] = 2
    wordtoix['<unk>'] = 3

    for idx, w in enumerate(vocab):
        wordtoix[w] = idx + 4
        ixtoword[idx+4] = w

    word_counts['<eos>'] = nsents
    word_counts['<bos>'] = nsents
    word_counts['<pad>'] = nsents
    word_counts['<unk>'] = nsents

    bias_init_vector = np.array([1.0 * word_counts[ ixtoword[i] ] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range

    return wordtoix, ixtoword, bias_init_vector


#######################################################################################################
# Parameters Setting
#######################################################################################################
batch_size = 5 # Being support batch_size
num_boxes = 50 # number of Detected regions in each image
feats_dim = 4096 # feature dimensions of each regions
project_dim = 1024 # project the features to one vector, which is 1024 dimensions

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words

S_max = 6
N_max = 50
T_stop = 0.5

n_epochs = 10
learning_rate = 0.0001


#######################################################################################################
# Word vocubulary and captions preprocessing stage
#######################################################################################################
img2paragraph = pickle.load(open('./img2paragraph', 'rb'))
all_sentences = []
for key, paragraph in img2paragraph.items():
    for each_sent in paragraph[1]:
        each_sent.replace(',', ' ,')
        all_sentences.append(each_sent)
word2idx, idx2word, bias_init_vector = preProBuildWordVocab(all_sentences, word_count_threshold=2)
np.save('./idx2word_batch', idx2word)

img2paragraph_modify = {}
for img_name, img_paragraph in img2paragraph.items():
    img_paragraph_1 = img_paragraph[1]

    # img_paragraph_1 is a list
    # it may contain the element: '' or ' ', like this:
    # [["a man is walking"], ["the dog is running"], [""], [" "]]
    # so, we should remove them ' ' and '' element
    if '' in img_paragraph_1:
        img_paragraph_1.remove('')
    if ' ' in paragraph[1]:
        img_paragraph_1.remove(' ')

    # the number sents in each paragraph
    # if the sents is bigger than S_max,
    # we force the number of sents to be S_max
    img_num_sents = len(img_paragraph_1)
    if img_num_sents > S_max:
        img_num_sents = S_max

    # if a paragraph has 4 sentences
    # then the img_num_distribution will be like this:
    # [0, 0, 0, 1, 1, 1]
    img_num_distribution = np.zeros([S_max], dtype=np.int32)
    img_num_distribution[img_num_sents-1:] = 1

    # we multiply the number 2, because the <pad> is encoded into 2
    img_captions_matrix = np.ones([S_max, N_max+1], dtype=np.int32) * 2 # zeros([6, 50])
    for idx, img_sent in enumerate(img_paragraph_1):
        # the number of sentences is img_num_sents
        if idx == img_num_sents:
            break

        # because we treat the ',' as a word
        img_sent = img_sent.replace(',', ' ,')

        # Because I have preprocess the paragraph_v1.json file in VScode before,
        # and I delete all the 2, 3, 4...bankspaces
        # so, actually, the 'elif' code will never run
        if img_sent[0] == ' ' and img_sent[1] != ' ':
            img_sent = img_sent[1:]
        elif img_sent[0] == ' ' and img_sent[1] == ' ' and img_sent[2] != ' ':
            img_sent = img_sent[2:]

        # Be careful the last part in a sentence, like this:
        # '...world.'
        # '...world. '
        if img_sent[-1] == '.':
            img_sent = img_sent[0:-1]
        elif img_sent[-1] == ' ' and img_sent[-2] == '.':
            img_sent = img_sent[0:-2]

        # Last, we add the <bos> and the <eos> in each sentences
        img_sent = '<bos> ' + img_sent + ' <eos>'

        # translate each word in a sentence into the unique number in word2idx dict
        # when we meet the word which is not in the word2idx dict, we use the mark: <unk>
        for idy, word in enumerate(img_sent.lower().split(' ')):
            # because the biggest number of words in a sentence is N_max, here is 50
            if idy == N_max:
                break

            if word in word2idx:
                img_captions_matrix[idx, idy] = word2idx[word]
            else:
                img_captions_matrix[idx, idy] = word2idx['<unk>']

    # Pay attention, the value type 'img_name' here is NUMBER, I change it to STRING type
    img2paragraph_modify[str(img_name)] = [img_num_distribution, img_captions_matrix]

with open('./img2paragraph_modify_batch', 'wb') as f:
    pickle.dump(img2paragraph_modify, f)


#######################################################################################################
# Train, validation and testing stage
#######################################################################################################
def train():
    ##############################################################################
    # some preparing work
    ##############################################################################
    model_path = './models_batch/'
    train_feats_path = './im2p_train_output.h5'
    train_output_file = h5py.File(train_feats_path, 'r')
    train_feats = train_output_file.get('feats')
    train_imgs_full_path_lists = open('./imgs_train_path.txt').read().splitlines()
    train_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists)


    # Model Initialization:
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                          batch_size = batch_size,
                                          num_boxes = num_boxes,
                                          feats_dim = feats_dim,
                                          project_dim = project_dim,
                                          sentRNN_lstm_dim = sentRNN_lstm_dim,
                                          sentRNN_FC_dim = sentRNN_FC_dim,
                                          wordRNN_lstm_dim = wordRNN_lstm_dim,
                                          S_max = S_max,
                                          N_max = N_max,
                                          word_embed_dim = word_embed_dim,
                                          bias_init_vector = bias_init_vector)

    tf_feats, tf_num_distribution, tf_captions_matrix, tf_captions_masks, tf_loss, tf_loss_sent, tf_loss_word = model.build_model()
    sess = tf.InteractiveSession()

    saver = tf.train.Saver(max_to_keep=500, write_version=1)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(tf_loss)
    tf.global_variables_initializer().run()

    # when you want to train the model from the front model
    #new_saver = tf.train.Saver(max_to_keep=500)
    #new_saver = tf.train.import_meta_graph('./models_batch/model-92.meta')
    #new_saver.restore(sess, tf.train.latest_checkpoint('./models_batch/'))

    all_vars = tf.trainable_variables()

    # open a loss file to record the loss value
    loss_fd = open('loss_batch.txt', 'w')
    img2idx = {}
    for idx, img in enumerate(train_imgs_names):
        img2idx[img] = idx

    # plt draw the loss curve
    # refer from: http://stackoverflow.com/questions/11874767/real-time-plotting-in-while-loop-with-matplotlib
    loss_to_draw = []

    for epoch in range(0, n_epochs):
        loss_to_draw_epoch = []
        # disorganize the order
        random.shuffle(train_imgs_names)

        for start, end in zip(range(0, len(train_imgs_names), batch_size),
                              range(batch_size, len(train_imgs_names), batch_size)):

            start_time = time.time()

            img_name = train_imgs_names[start:end]
            current_feats_index = map(lambda x: img2idx[x], img_name)
            current_feats = np.asarray( map(lambda x: train_feats[x], current_feats_index) )

            current_num_distribution = np.asarray( map(lambda x: img2paragraph_modify[x][0], img_name) )
            current_captions_matrix = np.asarray( map(lambda x: img2paragraph_modify[x][1], img_name) )

            current_captions_masks = np.zeros( (current_captions_matrix.shape[0], current_captions_matrix.shape[1], current_captions_matrix.shape[2]) )/idx2word_batch
            # find the non-zero element
            nonzeros = np.array( map(lambda each_matrix: np.array( map(lambda x: (x != 2).sum() + 1, each_matrix ) ), current_captions_matrix ) )
            for i in range(batch_size):
                for ind, row in enumerate(current_captions_masks[i]):
                    row[:(nonzeros[i, ind]-1)] = 1

            # shape of current_feats: batch_size x 50 x 4096
            # shape of current_num_distribution: batch_size x 6
            # shape of current_captions_matrix: batch_size x 6 x 50
            _, loss_val, loss_sent, loss_word= sess.run(
                                [train_op, tf_loss, tf_loss_sent, tf_loss_word],
                                feed_dict={
                                           tf_feats: current_feats,
                                           tf_num_distribution: current_num_distribution,
                                           tf_captions_matrix: current_captions_matrix,
                                           tf_captions_masks: current_captions_masks
                                })

            # append loss to list in a epoch
            loss_to_draw_epoch.append(loss_val)

            # running information
            print('idx: ', start, ' Epoch: ', epoch, ' loss: ', loss_val, ' loss_sent: ', loss_sent, ' loss_word: ', loss_word, ' Time cost: ', str((time.time() - start_time)))
            loss_fd.write('epoch ' + str(epoch) + ' loss ' + str(loss_val))

        # draw loss curve every epoch
        loss_to_draw.append(np.mean(loss_to_draw_epoch))
        plt_save_dir = './loss_imgs'
        plt_save_img_name = str(epoch) + '.png'
        plt.plot(range(len(loss_to_draw)), loss_to_draw, color='g')
        plt.grid(True)
        plt.savefig(os.path.join(plt_save_dir, plt_save_img_name))

        if np.mod(epoch, 10) == 0:
            print("Epoch ", epoch, " is done. Saving the model ...")
            saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
    loss_fd.close()


def test():
    
    start_time = time.time()
    # change the model path according to your environment
    model_path = './model-250'

    # It's very important to use Pandas to Series this idx2word dict
    # After this operation, we can use list to extract the word at the same time
    idx2word = pd.Series(np.load('./data/idx2word_batch.npy').tolist())

    test_feats_path = './data/im2p_test_output.h5'
    test_output_file = h5py.File(test_feats_path, 'r')
    test_feats = test_output_file.get('feats')

    test_imgs_full_path_lists = open('./densecap/imgs_test_path.txt').read().splitlines()
    test_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], test_imgs_full_path_lists)
    
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    test_model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                               batch_size = batch_size,
                                               num_boxes = num_boxes,
                                               feats_dim = feats_dim,
                                               project_dim = project_dim,
                                               sentRNN_lstm_dim = sentRNN_lstm_dim,
                                               sentRNN_FC_dim = sentRNN_FC_dim,
                                               wordRNN_lstm_dim = wordRNN_lstm_dim,
                                               S_max = S_max,
                                               N_max = N_max,
                                               word_embed_dim = word_embed_dim,
                                               bias_init_vector = bias_init_vector)
    

    tf_feats, tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors = test_model.generate_model()
    sess = tf.InteractiveSession()
    #print("Before there >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n\n")

    saver = tf.train.Saver()
    saver.restore(sess, model_path)

    img2idx = {}
    for idx, img in enumerate(test_imgs_names):
        img2idx[img] = idx
    print("IM2idx:::::::::::",img2idx)

    test_fd = open('HRNN_results.txt', 'w')
    for idx, img_name in enumerate(test_imgs_names):
        print(idx, img_name)
        test_fd.write(img_name + '\n')

        each_paragraph = []
        current_paragraph = ""

        current_feats_index = img2idx[img_name]
        current_feats = test_feats[current_feats_index]
        current_feats = np.reshape(current_feats, [1, 50, 4096])

        generated_paragraph_indexes, pred, sent_topic_vectors = sess.run(
                                                                         [tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors],
                                                                         feed_dict={
                                                                             tf_feats: current_feats
                                                                         })

        #generated_paragraph = idx2word[generated_paragraph_indexes]
        for sent_index in generated_paragraph_indexes:
            each_sent = []
            for word_index in sent_index:
                each_sent.append(idx2word[word_index])
            each_paragraph.append(each_sent)

        for idx, each_sent in enumerate(each_paragraph):
            # if the current sentence is the end sentence of the paragraph
            # According to the probability distribution:
            # CONTINUE: [1, 0]
            # STOP    : [0, 1]
            # So, if the first item of pred is less than the T_stop
            # the generation process is break
            if pred[idx][0][0] <= T_stop:
                break
            current_sent = ''
            for each_word in each_sent:
                current_sent += each_word + ' '
            current_sent = current_sent.replace('<eos> ', '')
            current_sent = current_sent.replace('<pad> ', '')
            current_sent = current_sent + '.'
            current_sent = current_sent.replace(' .', '.')
            current_sent = current_sent.replace(' ,', ',')
            current_paragraph +=current_sent
            if idx != len(each_paragraph) - 1:
                current_paragraph += ' '

        test_fd.write(current_paragraph + '\n')
    test_fd.close()
    print("Time cost: " + str(time.time()-start_time))




preprocessing word counts and creating vocab based on word count threshold 2
filtered words from 18418 to 9900


In [4]:
    #start by parts
    model_path = './models_batch/'
    train_feats_path = './im2p_val_output.h5'
    
    train_output_file = h5py.File(train_feats_path, 'r')
    #print("Train feats path--------------------------------------------->>>>>>>>",train_output_file)
    train_feats = train_output_file.get('feats')
    #print("Train feats->>>>>>>>>>>>>>>>",train_feats)
    train_imgs_full_path_lists = open('./imgs_val_path.txt').read().splitlines()
    #print("paths>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",train_imgs_full_path_lists)
    train_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists)
    print("Val images names",list(train_imgs_names))


    # Model Initialization:
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                          batch_size = batch_size,
                                          num_boxes = num_boxes,
                                          feats_dim = feats_dim,
                                          project_dim = project_dim,
                                          sentRNN_lstm_dim = sentRNN_lstm_dim,
                                          sentRNN_FC_dim = sentRNN_FC_dim,
                                          wordRNN_lstm_dim = wordRNN_lstm_dim,
                                          S_max = S_max,
                                          N_max = N_max,
                                          word_embed_dim = word_embed_dim,
                                          bias_init_vector = bias_init_vector)

    tf_feats, tf_num_distribution, tf_captions_matrix, tf_captions_masks, tf_loss, tf_loss_sent, tf_loss_word = model.build_model()
    sess = tf.InteractiveSession()
    
    
    model_path = './models_batch/'
    train_feats_path = './im2p_val_output.h5'
    #print("THE SESSION",)
    
    train_output_file = h5py.File(train_feats_path, 'r')
    #print("Train feats path--------------------------------------------->>>>>>>>",train_output_file)
    train_feats = train_output_file.get('feats')
    #print("Train feats->>>>>>>>>>>>>>>>",train_feats)
    train_imgs_full_path_lists = open('./imgs_val_path.txt').read().splitlines()
    #print("paths>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",train_imgs_full_path_lists)
    train_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists)
    #print("Train images names",list(train_imgs_names))
    
    #print("Train images names",list(train_imgs_names))
    

Val images names ['2385757', '2389264', '2407325', '2337141', '2345813', '2386694', '2368581', '2397809', '185', '2387911', '2343682', '2409165', '2357129', '498337', '2326621', '2320205', '2412782', '2352466', '2414097', '2381526', '2390981', '2375715', '2405096', '2402848', '2346948', '2387882', '2352243', '2337581', '2330919', '2380444', '2377611', '2354637', '2401996', '2391375', '2415510', '2400079', '2366081', '2360527', '2365490', '2336229', '2367475', '2373691', '2376356', '2368952', '2369653', '2355012', '2400892', '2369971', '2414408', '2394007', '2415978', '2405444', '2401822', '2387651', '2350622', '2318603', '2383962', '2377677', '2380510', '2392746', '2412229', '2392926', '2362383', '2398991', '2367805', '2360264', '2400991', '2354133', '2374619', '2413514', '2387844', '2349764', '2403572', '2372008', '2388111', '2366871', '2385053', '2400907', '2379868', '2389754', '2389615', '2365263', '2414350', '2409171', '4025', '2352062', '2378864', '2367278', '2319025', '2335091', 

In [3]:
sess.close()

In [5]:
    #saver = tf.train.Saver(max_to_keep=500, write_version=1)
    print("Learning Rate and Loss++++++++++++",(learning_rate,tf_loss))
    with tf.variable_scope('optimizer',reuse= tf.AUTO_REUSE):
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(tf_loss)
    print("Adam Done")
    tf.global_variables_initializer().run()
    print("After run")

Learning Rate and Loss++++++++++++ (0.0001, <tf.Tensor 'add_1224:0' shape=() dtype=float32>)
Adam Done
After run


In [6]:
all_vars = tf.trainable_variables()
print("Done",list(all_vars))

Done [<tf.Variable 'Wemb:0' shape=(9904, 1024) dtype=float32_ref>, <tf.Variable 'regionPooling_W:0' shape=(4096, 1024) dtype=float32_ref>, <tf.Variable 'regionPooling_b:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'logistic_Theta_W:0' shape=(512, 2) dtype=float32_ref>, <tf.Variable 'logistic_Theta_b:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'fc1_W:0' shape=(512, 1024) dtype=float32_ref>, <tf.Variable 'fc1_b:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'fc2_W:0' shape=(1024, 1024) dtype=float32_ref>, <tf.Variable 'fc2_b:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'embed_word_W:0' shape=(512, 9904) dtype=float32_ref>, <tf.Variable 'embed_word_b:0' shape=(9904,) dtype=float32_ref>, <tf.Variable 'sent_LSTM/basic_lstm_cell/kernel:0' shape=(1536, 2048) dtype=float32_ref>, <tf.Variable 'sent_LSTM/basic_lstm_cell/bias:0' shape=(2048,) dtype=float32_ref>, <tf.Variable 'word_LSTM/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(1536, 2048) dtype=float32_ref>, <tf.Varia

In [7]:
    model_path = './models_batch/'
    train_feats_path = './im2p_val_output.h5'
    
    train_output_file = h5py.File(train_feats_path, 'r')
    #print("Train feats path--------------------------------------------->>>>>>>>",train_output_file)
    train_feats = train_output_file.get('feats')
    #print("Train feats->>>>>>>>>>>>>>>>",train_feats)
    train_imgs_full_path_lists = open('./imgs_val_path.txt').read().splitlines()
    #print("paths>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",train_imgs_full_path_lists)
    train_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists)
    #print("Train images names",list(train_imgs_names))
    
    print("Train images names",list(train_imgs_names))

Train images names ['2385757', '2389264', '2407325', '2337141', '2345813', '2386694', '2368581', '2397809', '185', '2387911', '2343682', '2409165', '2357129', '498337', '2326621', '2320205', '2412782', '2352466', '2414097', '2381526', '2390981', '2375715', '2405096', '2402848', '2346948', '2387882', '2352243', '2337581', '2330919', '2380444', '2377611', '2354637', '2401996', '2391375', '2415510', '2400079', '2366081', '2360527', '2365490', '2336229', '2367475', '2373691', '2376356', '2368952', '2369653', '2355012', '2400892', '2369971', '2414408', '2394007', '2415978', '2405444', '2401822', '2387651', '2350622', '2318603', '2383962', '2377677', '2380510', '2392746', '2412229', '2392926', '2362383', '2398991', '2367805', '2360264', '2400991', '2354133', '2374619', '2413514', '2387844', '2349764', '2403572', '2372008', '2388111', '2366871', '2385053', '2400907', '2379868', '2389754', '2389615', '2365263', '2414350', '2409171', '4025', '2352062', '2378864', '2367278', '2319025', '2335091'

In [8]:
    model_path = './models_batch/'
    train_feats_path = './im2p_val_output.h5'
    
    train_output_file = h5py.File(train_feats_path, 'r')
    #print("Train feats path--------------------------------------------->>>>>>>>",train_output_file)
    train_feats = train_output_file.get('feats')
    #print("Train feats->>>>>>>>>>>>>>>>",train_feats)
    train_imgs_full_path_lists = open('./imgs_val_path.txt').read().splitlines()
    #print("paths>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",train_imgs_full_path_lists)
    train_imgs_names = list(map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists))
    #print("Train images names",list(train_imgs_names))
    
    #print("Train images names",list(train_imgs_names))
    
#     loss_fd = open('loss_batch.txt', 'w')
    img2idx = {}
    print("train_imgs_names-------",train_imgs_names)
    for idx, img in enumerate(train_imgs_names):
        img2idx[img] = idx
    print("IMG@IDX",img2idx)

    # plt draw the loss curve
    # refer from: http://stackoverflow.com/questions/11874767/real-time-plotting-in-while-loop-with-matplotlib
    loss_to_draw = []

train_imgs_names------- ['2385757', '2389264', '2407325', '2337141', '2345813', '2386694', '2368581', '2397809', '185', '2387911', '2343682', '2409165', '2357129', '498337', '2326621', '2320205', '2412782', '2352466', '2414097', '2381526', '2390981', '2375715', '2405096', '2402848', '2346948', '2387882', '2352243', '2337581', '2330919', '2380444', '2377611', '2354637', '2401996', '2391375', '2415510', '2400079', '2366081', '2360527', '2365490', '2336229', '2367475', '2373691', '2376356', '2368952', '2369653', '2355012', '2400892', '2369971', '2414408', '2394007', '2415978', '2405444', '2401822', '2387651', '2350622', '2318603', '2383962', '2377677', '2380510', '2392746', '2412229', '2392926', '2362383', '2398991', '2367805', '2360264', '2400991', '2354133', '2374619', '2413514', '2387844', '2349764', '2403572', '2372008', '2388111', '2366871', '2385053', '2400907', '2379868', '2389754', '2389615', '2365263', '2414350', '2409171', '4025', '2352062', '2378864', '2367278', '2319025', '233

In [10]:
#sess = tf.InteractiveSession()
#tf.reset_default_graph()
saver = tf.train.import_meta_graph('./models_batch/model-100.meta')
saver.restore(sess, tf.train.latest_checkpoint('./models_batch/'))
for epoch in range(101, n_epochs):
        loss_to_draw_epoch = []
        #saver = tf.train.import_meta_graph('./models_batch/model-0.meta')
        #saver.restore(sess, tf.train.latest_checkpoint('./models_batch/'))
        # disorganize the order
        #print("train images",train_imgs_names,type(train_imgs_names))
        #train_imgs_names=list(train_imgs_names)
        random.shuffle(train_imgs_names)

        for start, end in zip(range(0, len(train_imgs_names), batch_size),
                              range(batch_size, len(train_imgs_names), batch_size)):
            loss_fd = open('loss_batch.txt', 'a')
            start_time = time.time()
            print("Start and end",(start,end))
            img_name = train_imgs_names[start:end]
            current_feats_index = map(lambda x: img2idx[x], img_name)
            current_feats = np.asarray( list(map(lambda x: train_feats[x], current_feats_index) ))


            current_num_distribution = np.asarray(list(map(lambda x: img2paragraph_modify[x][0], img_name)))
            current_captions_matrix = np.asarray(list(map(lambda x: img2paragraph_modify[x][1], img_name)))

            #print("Blah Blah",current_captions_matrix,type(current_captions_matrix))
            
            
            #print("Current captions matrix >>>>>>>>>>>>>>>>>>",current_captions_matrix,type(current_captions_matrix))
            #print("kwiojwejiowe",(current_captions_matrix.shape)[0])
            
            current_captions_masks = np.zeros( (current_captions_matrix.shape[0], current_captions_matrix.shape[1], current_captions_matrix.shape[2]) )
            #done by me current_captions_masks = np.zeros( (current_captions_matrix.shape[0], current_captions_matrix.shape[1], current_captions_matrix.shape[2]) )/idx2word_batch
            # find the non-zero element
            #done by me nonzeros = np.asarray(map(lambda each_matrix: np.asarray( map(lambda x: (x != 2).sum() + 1, each_matrix ) ), current_captions_matrix ) )
            nonzeros = np.array(list(map(lambda each_matrix: np.array( list(map(lambda x: (x != 2).sum() + 1, each_matrix ) )), current_captions_matrix ) ))

            #print("NONZEROS",nonzeros,nonzeros.shape)
            for i in range(batch_size):
                for ind, row in enumerate(current_captions_masks[i]):
                    row[:(nonzeros[i, ind]-1)] = 1

            # shape of current_feats: batch_size x 50 x 4096
            # shape of current_num_distribution: batch_size x 6
            # shape of current_captions_matrix: batch_size x 6 x 50
            #print("Done w the loop")
            #print("train op, tf loss ,tf_loss_sent,tf_loss_word",[train_op, tf_loss, tf_loss_sent, tf_loss_word])
            _, loss_val, loss_sent, loss_word= sess.run(
                                [train_op, tf_loss, tf_loss_sent, tf_loss_word],
                                feed_dict={
                                           tf_feats: current_feats,
                                           tf_num_distribution: current_num_distribution,
                                           tf_captions_matrix: current_captions_matrix,
                                           tf_captions_masks: current_captions_masks
                                })

            # append loss to list in a epoch
            loss_to_draw_epoch.append(loss_val)

            # running information
            print('idx: ', start, ' Epoch: ', epoch, ' loss: ', loss_val, ' loss_sent: ', loss_sent, ' loss_word: ', loss_word, ' Time cost: ', str((time.time() - start_time)))
            
            loss_fd.write('start: '+ str(start) +' end: ' + str(end) +' epoch ' + str(epoch) + ' loss ' + str(loss_val) + '\n')
            loss_fd.close()

        # draw loss curve every epoch
        loss_to_draw.append(np.mean(loss_to_draw_epoch))
        plt_save_dir = './loss_imgs'
        plt_save_img_name = str(epoch) + '.png'
        plt.plot(range(len(loss_to_draw)), loss_to_draw, color='g')
        plt.grid(True)
        plt.savefig(os.path.join(plt_save_dir, plt_save_img_name))

        if np.mod(epoch, 1) == 0:
            print("Epoch ", epoch, " is done. Saving the model ...")
            saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
#loss_fd.close()

sess.close()

TypeError: Cannot interpret feed_dict key as Tensor: The name 'save/Const:0' refers to a Tensor which does not exist. The operation, 'save/Const', does not exist in the graph.

In [1]:
#!/usr/bin/env python
# coding=utf-8

__author__ = "Xinpeng.Chen"

import os
import sys
import time
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import random

import h5py
import ipdb

import tensorflow as tf


# ------------------------------------------------------------------------------------------------------
# Initialization class
#  1. Pooling the visual features into a single dense feature
#  2. Then, build sentence LSTM, word LSTM
# ------------------------------------------------------------------------------------------------------
class RegionPooling_HierarchicalRNN():
    def __init__(self, n_words,
                       batch_size,
                       num_boxes,
                       feats_dim,
                       project_dim,
                       sentRNN_lstm_dim,
                       sentRNN_FC_dim,
                       wordRNN_lstm_dim,
                       S_max,
                       N_max,
                       word_embed_dim,
                       bias_init_vector=None):

        self.n_words = n_words
        self.batch_size = batch_size
        self.num_boxes = num_boxes # 50
        self.feats_dim = feats_dim # 4096
        self.project_dim = project_dim # 1024
        self.S_max = S_max # 6
        self.N_max = N_max # 50
        self.word_embed_dim = word_embed_dim # 1024

        self.sentRNN_lstm_dim = sentRNN_lstm_dim # 512 hidden size
        self.sentRNN_FC_dim = sentRNN_FC_dim # 1024 in fully connected layer
        self.wordRNN_lstm_dim = wordRNN_lstm_dim # 512 hidden size
	


        # word embedding, parameters of embedding
        # embedding shape: n_words x wordRNN_lstm_dim
        with tf.device('/cpu:0'):
            self.Wemb = tf.Variable(tf.random_uniform([n_words, word_embed_dim], -0.1, 0.1), name='Wemb')
        print("Wemb output>>>>>>>>>>>>",self.Wemb,tf.shape(self.Wemb))
        #self.bemb = tf.Variable(tf.zeros([word_embed_dim]), name='bemb')

        # regionPooling_W shape: 4096 x 1024
        # regionPooling_b shape: 1024
        self.regionPooling_W = tf.Variable(tf.random_uniform([feats_dim, project_dim], -0.1, 0.1), name='regionPooling_W')
        self.regionPooling_b = tf.Variable(tf.zeros([project_dim]), name='regionPooling_b')

        # sentence LSTM
        self.sent_LSTM = tf.nn.rnn_cell.BasicLSTMCell(sentRNN_lstm_dim, state_is_tuple=True)

        # logistic classifier
        self.logistic_Theta_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, 2], -0.1, 0.1), name='logistic_Theta_W')
        self.logistic_Theta_b = tf.Variable(tf.zeros(2), name='logistic_Theta_b')

        # fc1_W: 512 x 1024, fc1_b: 1024
        # fc2_W: 1024 x 1024, fc2_b: 1024
        self.fc1_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, sentRNN_FC_dim], -0.1, 0.1), name='fc1_W')
        self.fc1_b = tf.Variable(tf.zeros(sentRNN_FC_dim), name='fc1_b')
        self.fc2_W = tf.Variable(tf.random_uniform([sentRNN_FC_dim, 1024], -0.1, 0.1), name='fc2_W')
        self.fc2_b = tf.Variable(tf.zeros(1024), name='fc2_b')
        def get_a_cell(lstm_size):
          lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
          return lstm
        # word LSTM
        self.word_LSTM = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)
        #self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([self.word_LSTM] * 2, state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([get_a_cell(wordRNN_lstm_dim) for i in range(2)], state_is_tuple=True)
        #self.word_LSTM2 = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)


        self.embed_word_W = tf.Variable(tf.random_uniform([wordRNN_lstm_dim, n_words], -0.1,0.1), name='embed_word_W')
        if bias_init_vector is not None:
            self.embed_word_b = tf.Variable(bias_init_vector.astype(np.float32), name='embed_word_b')
        else:
            self.embed_word_b = tf.Variable(tf.zeros([n_words]), name='embed_word_b')

    def build_model(self):
        # receive the feats in the current image
        # it's shape is 10 x 50 x 4096
        # tmp_feats: 500 x 4096
        feats = tf.placeholder(tf.float32, [self.batch_size, self.num_boxes, self.feats_dim])
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 500 x 4096 * 4096 x 1024 --> 500 x 1024
        # project_vec: 10 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [self.batch_size, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # receive the [continue:0, stop:1] lists
        # example: [0, 0, 0, 0, 1, 1], it means this paragraph has five sentences
        num_distribution = tf.placeholder(tf.int32, [self.batch_size, self.S_max])

        # receive the ground truth words, which has been changed to idx use word2idx function
        captions = tf.placeholder(tf.int32, [self.batch_size, self.S_max, self.N_max+1])
        #print("Captions:>>>>>>>>>>>>>>>>>>>>>>>",captions)
        captions_masks = tf.placeholder(tf.float32, [self.batch_size, self.S_max, self.N_max+1])

        # ---------------------------------------------------------------------------------------------------------------------
        # The method which initialize the state, is refered from below sites:
        # 1. http://stackoverflow.com/questions/38241410/tensorflow-remember-lstm-state-for-next-batch-stateful-lstm/38417699
        # 2. https://www.tensorflow.org/api_docs/python/rnn_cell/classes_storing_split_rnncell_state#LSTMStateTuple
        # 3. https://medium.com/@erikhallstrm/using-the-tensorflow-lstm-api-3-7-5f2b97ca6b73#.u4w9z6h0h
        # ---------------------------------------------------------------------------------------------------------------------
        sent_state = self.sent_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state = self.word_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state1 = self.word_LSTM1.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state2 = self.word_LSTM2.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #sent_state = tf.zeros([self.batch_size, self.sent_LSTM1.state_size])
        #word_state1 = tf.zeros([self.batch_size, self.word_LSTM1.state_size])
        #word_state2 = tf.zeros([self.batch_size, self.word_LSTM2.state_size])

        probs = []
        loss = 0.0
        loss_sent = 0.0
        loss_word = 0.0
        lambda_sent = 5.0
        lambda_word = 1.0

        print('Start build model:')
        #----------------------------------------------------------------------------------------------
        # Hierarchical RNN: sentence RNN and words RNN
        # The word RNN has the max number, N_max = 50, the number in the papar is 50
        #----------------------------------------------------------------------------------------------
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            with tf.variable_scope('sent_LSTM',reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            # sent_state is a tuple, sent_state = (c, h)
            # 'c': shape=(1, 512) dtype=float32, 'h': shape=(1, 512) dtype=float32
            # The loss here, I refer from the web which is very helpful for me:
            # 1. http://stackoverflow.com/questions/34240703/difference-between-tensorflow-tf-nn-softmax-and-tf-nn-softmax-cross-entropy-with
            # 2. http://stackoverflow.com/questions/35277898/tensorflow-for-binary-classification
            # 3. http://stackoverflow.com/questions/35226198/is-this-one-hot-encoding-in-tensorflow-fast-or-flawed-for-any-reason
            # 4. http://stackoverflow.com/questions/35198528/reshape-y-train-for-binary-text-classification-in-tensorflow
            sentRNN_logistic_mu = tf.nn.xw_plus_b( sent_output, self.logistic_Theta_W, self.logistic_Theta_b )
            sentRNN_label = tf.stack([ 1 - num_distribution[:, i], num_distribution[:, i] ])
            sentRNN_label = tf.transpose(sentRNN_label)
            sentRNN_loss = tf.nn.softmax_cross_entropy_with_logits(logits=sentRNN_logistic_mu, labels=sentRNN_label)
            sentRNN_loss = tf.reduce_sum(sentRNN_loss)/self.batch_size
            loss += sentRNN_loss * lambda_sent
            loss_sent += sentRNN_loss

            # the begining input of word_LSTM is topic vector, and DON'T compute the loss
            # This is follow the paper: Show and Tell
            #word_state = self.word_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
            #with tf.variable_scope('word_LSTM'):
            #    word_output, word_state = self.word_LSTM(sent_topic_vec)
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, captions[:, i, j])

                with tf.variable_scope('word_LSTM',reuse=tf.AUTO_REUSE):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                # How to make one-hot encoder, I refer from this excellent web:
                # http://stackoverflow.com/questions/33681517/tensorflow-one-hot-encoder
                labels = tf.reshape(captions[:, i, j+1], [-1, 1])
                #print("Labels and its shape +++++++++++++++",labels,tf.shape(labels))
                indices = tf.reshape(tf.range(0, self.batch_size, 1), [-1, 1])
                #print("Indices and its shape +++++++++++++++",indices,tf.shape(indices))
                concated = tf.concat([indices, labels],1)
                #print("Concated+++++++++++++++++++",concated)
                print("Success")
                onehot_labels = tf.sparse_to_dense(concated, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)

                # At each timestep the hidden state of the last LSTM layer is used to predict a distribution
                # over the words in the vocbulary
                logit_words = tf.nn.xw_plus_b(word_output[:], self.embed_word_W, self.embed_word_b)
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=onehot_labels)
                cross_entropy = cross_entropy * captions_masks[:, i, j]
                loss_wordRNN = tf.reduce_sum(cross_entropy) / self.batch_size
                loss += loss_wordRNN * lambda_word
                loss_word += loss_wordRNN

        return feats, num_distribution, captions, captions_masks, loss, loss_sent, loss_word

    def generate_model(self):
        # feats: 1 x 50 x 4096
        feats = tf.placeholder(tf.float32, [1, self.num_boxes, self.feats_dim])
        # tmp_feats: 50 x 4096
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 50 x 4096 * 4096 x 1024 + 1024 --> 50 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [1, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # initialize the sent_LSTM state
        sent_state = self.sent_LSTM.zero_state(batch_size=1, dtype=tf.float32)

        # save the generated paragraph to list, here I named generated_sents
        generated_paragraph = []

        # pred
        pred_re = []

        # T_stop: run the sentence RNN forward until the stopping probability p_i (STOP) exceeds a threshold T_stop
        T_stop = tf.constant(0.5)

        # Start build the generation model
        print('Start build the generation model: ')

        # sentence RNN
        #word_state = self.word_LSTM.zero_state(batch_size=1, dtype=tf.float32)
        #with tf.variable_scope('word_LSTM'):
        #    word_output, word_state = self.word_LSTM(sent_topic_vec, word_state)
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            # sent_state:
            # LSTMStateTuple(c=<tf.Tensor 'sent_LSTM/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
            #                h=<tf.Tensor 'sent_LSTM/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>)
            with tf.variable_scope('sent_LSTM',reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            # self.fc1_W: 512 x 1024, self.fc1_b: 1024
            # hidden1: 1 x 1024
            # sent_topic_vec: 1 x 1024
            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            sentRNN_logistic_mu = tf.nn.xw_plus_b(sent_output, self.logistic_Theta_W, self.logistic_Theta_b)
            pred = tf.nn.softmax(sentRNN_logistic_mu)
            pred_re.append(pred)

            # save the generated sentence to list, named generated_sent
            generated_sent = []

            # initialize the word LSTM state
            #word_state = self.word_LSTM.zero_state(batch_size=1, dtype=tf.float32)
            #with tf.variable_scope('word_LSTM'):
            #    word_output, word_state = self.word_LSTM(sent_topic_vec, word_state)
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            # word RNN, unrolled to N_max time steps
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                if j == 0:
                    with tf.device('/cpu:0'):
                        # get word embedding of BOS (index = 0)
                        current_embed = tf.nn.embedding_lookup(self.Wemb, tf.zeros([1], dtype=tf.int64))

                with tf.variable_scope('word_LSTM',reuse=tf.AUTO_REUSE):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                # word_state:
                # (
                #     LSTMStateTuple(c=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell0/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
                #                    h=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell0/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>),
                #     LSTMStateTuple(c=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell1/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
                #                    h=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>)
                # )
                logit_words = tf.nn.xw_plus_b(word_output, self.embed_word_W, self.embed_word_b)
                max_prob_index = tf.argmax(logit_words, 1)[0]
                generated_sent.append(max_prob_index)

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, max_prob_index)
                    current_embed = tf.expand_dims(current_embed, 0)

            generated_paragraph.append(generated_sent)

        return feats, generated_paragraph, pred_re, sent_topic_vec


# -----------------------------------------------------------------------------------------------------
# Preparing Functions
# -----------------------------------------------------------------------------------------------------
def preProBuildWordVocab(sentence_iterator, word_count_threshold=5):
    # borrowed this function from NeuralTalk
    print('preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, ))

    word_counts = {}
    nsents = 0

    for sent in sentence_iterator:
        nsents += 1
        tmp_sent = sent.lower().split(' ')
        if '' in tmp_sent:
            tmp_sent.remove('')

        for w in tmp_sent:
           word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print('filtered words from %d to %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '<bos>'
    ixtoword[1] = '<eos>'
    ixtoword[2] = '<pad>'
    ixtoword[3] = '<unk>'

    wordtoix = {}
    wordtoix['<bos>'] = 0
    wordtoix['<eos>'] = 1
    wordtoix['<pad>'] = 2
    wordtoix['<unk>'] = 3

    for idx, w in enumerate(vocab):
        wordtoix[w] = idx + 4
        ixtoword[idx+4] = w

    word_counts['<eos>'] = nsents
    word_counts['<bos>'] = nsents
    word_counts['<pad>'] = nsents
    word_counts['<unk>'] = nsents

    bias_init_vector = np.array([1.0 * word_counts[ ixtoword[i] ] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range

    return wordtoix, ixtoword, bias_init_vector


#######################################################################################################
# Parameters Setting
#######################################################################################################
batch_size = 5 # Being support batch_size
num_boxes = 50 # number of Detected regions in each image
feats_dim = 4096 # feature dimensions of each regions
project_dim = 1024 # project the features to one vector, which is 1024 dimensions

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words

S_max = 6
N_max = 50
T_stop = 0.5

n_epochs = 500
learning_rate = 0.0001


#######################################################################################################
# Word vocubulary and captions preprocessing stage
#######################################################################################################
img2paragraph = pickle.load(open('./img2paragraph', 'rb'))
all_sentences = []
for key, paragraph in img2paragraph.items():
    for each_sent in paragraph[1]:
        each_sent.replace(',', ' ,')
        all_sentences.append(each_sent)
word2idx, idx2word, bias_init_vector = preProBuildWordVocab(all_sentences, word_count_threshold=2)
np.save('./idx2word_batch', idx2word)

img2paragraph_modify = {}
for img_name, img_paragraph in img2paragraph.items():
    img_paragraph_1 = img_paragraph[1]

    # img_paragraph_1 is a list
    # it may contain the element: '' or ' ', like this:
    # [["a man is walking"], ["the dog is running"], [""], [" "]]
    # so, we should remove them ' ' and '' element
    if '' in img_paragraph_1:
        img_paragraph_1.remove('')
    if ' ' in paragraph[1]:
        img_paragraph_1.remove(' ')

    # the number sents in each paragraph
    # if the sents is bigger than S_max,
    # we force the number of sents to be S_max
    img_num_sents = len(img_paragraph_1)
    if img_num_sents > S_max:
        img_num_sents = S_max

    # if a paragraph has 4 sentences
    # then the img_num_distribution will be like this:
    # [0, 0, 0, 1, 1, 1]
    img_num_distribution = np.zeros([S_max], dtype=np.int32)
    img_num_distribution[img_num_sents-1:] = 1

    # we multiply the number 2, because the <pad> is encoded into 2
    img_captions_matrix = np.ones([S_max, N_max+1], dtype=np.int32) * 2 # zeros([6, 50])
    for idx, img_sent in enumerate(img_paragraph_1):
        # the number of sentences is img_num_sents
        if idx == img_num_sents:
            break

        # because we treat the ',' as a word
        img_sent = img_sent.replace(',', ' ,')

        # Because I have preprocess the paragraph_v1.json file in VScode before,
        # and I delete all the 2, 3, 4...bankspaces
        # so, actually, the 'elif' code will never run
        if img_sent[0] == ' ' and img_sent[1] != ' ':
            img_sent = img_sent[1:]
        elif img_sent[0] == ' ' and img_sent[1] == ' ' and img_sent[2] != ' ':
            img_sent = img_sent[2:]

        # Be careful the last part in a sentence, like this:
        # '...world.'
        # '...world. '
        if img_sent[-1] == '.':
            img_sent = img_sent[0:-1]
        elif img_sent[-1] == ' ' and img_sent[-2] == '.':
            img_sent = img_sent[0:-2]

        # Last, we add the <bos> and the <eos> in each sentences
        img_sent = '<bos> ' + img_sent + ' <eos>'

        # translate each word in a sentence into the unique number in word2idx dict
        # when we meet the word which is not in the word2idx dict, we use the mark: <unk>
        for idy, word in enumerate(img_sent.lower().split(' ')):
            # because the biggest number of words in a sentence is N_max, here is 50
            if idy == N_max:
                break

            if word in word2idx:
                img_captions_matrix[idx, idy] = word2idx[word]
            else:
                img_captions_matrix[idx, idy] = word2idx['<unk>']

    # Pay attention, the value type 'img_name' here is NUMBER, I change it to STRING type
    img2paragraph_modify[str(img_name)] = [img_num_distribution, img_captions_matrix]

with open('./img2paragraph_modify_batch', 'wb') as f:
    pickle.dump(img2paragraph_modify, f)


#######################################################################################################
# Train, validation and testing stage
#######################################################################################################
def train():
    ##############################################################################
    # some preparing work
    ##############################################################################
    model_path = './models_batch/'
    train_feats_path = './im2p_val_output.h5'
    train_output_file = h5py.File(train_feats_path, 'r')
    train_feats = train_output_file.get('feats')
    train_imgs_full_path_lists = open('./imgs_val_path.txt').read().splitlines()
    train_imgs_names = list(map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists))
    print("Train Images Names",train_imgs_names)

    # Model Initialization:
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                          batch_size = batch_size,
                                          num_boxes = num_boxes,
                                          feats_dim = feats_dim,
                                          project_dim = project_dim,
                                          sentRNN_lstm_dim = sentRNN_lstm_dim,
                                          sentRNN_FC_dim = sentRNN_FC_dim,
                                          wordRNN_lstm_dim = wordRNN_lstm_dim,
                                          S_max = S_max,
                                          N_max = N_max,
                                          word_embed_dim = word_embed_dim,
                                          bias_init_vector = bias_init_vector)

    tf_feats, tf_num_distribution, tf_captions_matrix, tf_captions_masks, tf_loss, tf_loss_sent, tf_loss_word = model.build_model()
    sess = tf.Session()

    saver = tf.train.Saver(max_to_keep=500, write_version=1)
    with tf.variable_scope('optimizer',reuse= tf.AUTO_REUSE):
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(tf_loss)
    tf.global_variables_initializer().run(session=sess)
    print("Done w session")

    # when you want to train the model from the front model
    #new_saver = tf.train.Saver(max_to_keep=500)
    #new_saver = tf.train.import_meta_graph('./models_batch/model-92.meta')
    #new_saver.restore(sess, tf.train.latest_checkpoint('./models_batch/'))

    all_vars = tf.trainable_variables()

    # open a loss file to record the loss value
    #loss_fd = open('loss_batch.txt', 'a')
    img2idx = {}
    for idx, img in enumerate(train_imgs_names):
        img2idx[img] = idx
    print("Img2idx",img2idx)

    # plt draw the loss curve
    # refer from: http://stackoverflow.com/questions/11874767/real-time-plotting-in-while-loop-with-matplotlib
    loss_to_draw = []
    print("Before epoch loop->>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    saver = tf.train.import_meta_graph('./models_batch/model-1.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./models_batch/'))
    for epoch in range(2, n_epochs):
        loss_to_draw_epoch = []
        # disorganize the order
        random.shuffle(train_imgs_names)

        for start, end in zip(range(0, len(train_imgs_names), batch_size),
                              range(batch_size, len(train_imgs_names), batch_size)):
            loss_fd = open('loss_batch.txt', 'a')

            start_time = time.time()
            print("Start and end",(start,end))
            img_name = train_imgs_names[start:end]
            current_feats_index = map(lambda x: img2idx[x], img_name)
            current_feats = np.asarray( list(map(lambda x: train_feats[x], current_feats_index) ))

            current_num_distribution = np.asarray( list(map(lambda x: img2paragraph_modify[x][0], img_name) ))
            current_captions_matrix = np.asarray( list(map(lambda x: img2paragraph_modify[x][1], img_name) ))

            current_captions_masks = np.zeros( (current_captions_matrix.shape[0], current_captions_matrix.shape[1], current_captions_matrix.shape[2]) )
            # find the non-zero element
            nonzeros = np.array( list(map(lambda each_matrix: np.array( list(map(lambda x: (x != 2).sum() + 1, each_matrix ) )), current_captions_matrix ) ))
            for i in range(batch_size):
                for ind, row in enumerate(current_captions_masks[i]):
                    row[:(nonzeros[i, ind]-1)] = 1

            # shape of current_feats: batch_size x 50 x 4096
            # shape of current_num_distribution: batch_size x 6
            # shape of current_captions_matrix: batch_size x 6 x 50
            _, loss_val, loss_sent, loss_word= sess.run(
                                [train_op, tf_loss, tf_loss_sent, tf_loss_word],
                                feed_dict={
                                           tf_feats: current_feats,
                                           tf_num_distribution: current_num_distribution,
                                           tf_captions_matrix: current_captions_matrix,
                                           tf_captions_masks: current_captions_masks
                                })

            # append loss to list in a epoch
            loss_to_draw_epoch.append(loss_val)

            # running information
            print('idx: ', start, ' Epoch: ', epoch, ' loss: ', loss_val, ' loss_sent: ', loss_sent, ' loss_word: ', loss_word, ' Time cost: ', str((time.time() - start_time)))
            loss_fd.write('start: '+ str(start) +' end: ' + str(end) +' epoch ' + str(epoch) + ' loss ' + str(loss_val) + '\n')
            loss_fd.close()
        # draw loss curve every epoch
        loss_to_draw.append(np.mean(loss_to_draw_epoch))
        plt_save_dir = './loss_imgs'
        plt_save_img_name = str(epoch) + '.png'
        plt.plot(range(len(loss_to_draw)), loss_to_draw, color='g')
        plt.grid(True)
        plt.savefig(os.path.join(plt_save_dir, plt_save_img_name))

        if np.mod(epoch, 1) == 0:
            print("Epoch ", epoch, " is done. Saving the model ...")
            saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
    #loss_fd.close()


def test():
    
    start_time = time.time()
    # change the model path according to your environment
    model_path = './model-250'

    # It's very important to use Pandas to Series this idx2word dict
    # After this operation, we can use list to extract the word at the same time
    idx2word = pd.Series(np.load('./data/idx2word_batch.npy').tolist())

    test_feats_path = './data/im2p_test_output.h5'
    test_output_file = h5py.File(test_feats_path, 'r')
    test_feats = test_output_file.get('feats')

    test_imgs_full_path_lists = open('./densecap/imgs_test_path.txt').read().splitlines()
    test_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], test_imgs_full_path_lists)
    
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    test_model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                               batch_size = batch_size,
                                               num_boxes = num_boxes,
                                               feats_dim = feats_dim,
                                               project_dim = project_dim,
                                               sentRNN_lstm_dim = sentRNN_lstm_dim,
                                               sentRNN_FC_dim = sentRNN_FC_dim,
                                               wordRNN_lstm_dim = wordRNN_lstm_dim,
                                               S_max = S_max,
                                               N_max = N_max,
                                               word_embed_dim = word_embed_dim,
                                               bias_init_vector = bias_init_vector)
    

    tf_feats, tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors = test_model.generate_model()
    sess = tf.InteractiveSession()
    #print("Before there >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n\n")

    saver = tf.train.Saver()
    saver.restore(sess, model_path)

    img2idx = {}
    for idx, img in enumerate(test_imgs_names):
        img2idx[img] = idx
    print("IM2idx:::::::::::",img2idx)

    test_fd = open('HRNN_results.txt', 'w')
    for idx, img_name in enumerate(test_imgs_names):
        print(idx, img_name)
        test_fd.write(img_name + '\n')

        each_paragraph = []
        current_paragraph = ""

        current_feats_index = img2idx[img_name]
        current_feats = test_feats[current_feats_index]
        current_feats = np.reshape(current_feats, [1, 50, 4096])

        generated_paragraph_indexes, pred, sent_topic_vectors = sess.run(
                                                                         [tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors],
                                                                         feed_dict={
                                                                             tf_feats: current_feats
                                                                         })

        #generated_paragraph = idx2word[generated_paragraph_indexes]
        for sent_index in generated_paragraph_indexes:
            each_sent = []
            for word_index in sent_index:
                each_sent.append(idx2word[word_index])
            each_paragraph.append(each_sent)

        for idx, each_sent in enumerate(each_paragraph):
            # if the current sentence is the end sentence of the paragraph
            # According to the probability distribution:
            # CONTINUE: [1, 0]
            # STOP    : [0, 1]
            # So, if the first item of pred is less than the T_stop
            # the generation process is break
            if pred[idx][0][0] <= T_stop:
                break
            current_sent = ''
            for each_word in each_sent:
                current_sent += each_word + ' '
            current_sent = current_sent.replace('<eos> ', '')
            current_sent = current_sent.replace('<pad> ', '')
            current_sent = current_sent + '.'
            current_sent = current_sent.replace(' .', '.')
            current_sent = current_sent.replace(' ,', ',')
            current_paragraph +=current_sent
            if idx != len(each_paragraph) - 1:
                current_paragraph += ' '

        test_fd.write(current_paragraph + '\n')
    test_fd.close()
    print("Time cost: " + str(time.time()-start_time))

train()


preprocessing word counts and creating vocab based on word count threshold 2
filtered words from 18418 to 9900
Train Images Names ['2385757', '2389264', '2407325', '2337141', '2345813', '2386694', '2368581', '2397809', '185', '2387911', '2343682', '2409165', '2357129', '498337', '2326621', '2320205', '2412782', '2352466', '2414097', '2381526', '2390981', '2375715', '2405096', '2402848', '2346948', '2387882', '2352243', '2337581', '2330919', '2380444', '2377611', '2354637', '2401996', '2391375', '2415510', '2400079', '2366081', '2360527', '2365490', '2336229', '2367475', '2373691', '2376356', '2368952', '2369653', '2355012', '2400892', '2369971', '2414408', '2394007', '2415978', '2405444', '2401822', '2387651', '2350622', '2318603', '2383962', '2377677', '2380510', '2392746', '2412229', '2392926', '2362383', '2398991', '2367805', '2360264', '2400991', '2354133', '2374619', '2413514', '2387844', '2349764', '2403572', '2372008', '2388111', '2366871', '2385053', '2400907', '2379868', '2389

W0711 14:30:03.287303 140399233443584 deprecation.py:323] From <ipython-input-1-ddcb88721579>:68: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0711 14:30:03.311513 140399233443584 deprecation.py:323] From <ipython-input-1-ddcb88721579>:86: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0711 14:30:03.426924 140399233443584 deprecation.py:506] From /home/student/anaconda3/envs/CPU/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future versio

Wemb output>>>>>>>>>>>> <tf.Variable 'Wemb:0' shape=(9904, 1024) dtype=float32_ref> Tensor("Shape:0", shape=(2,), dtype=int32)
Start build model:


W0711 14:30:03.778566 140399233443584 deprecation.py:323] From <ipython-input-1-ddcb88721579>:166: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0711 14:30:04.040371 140399233443584 deprecation.py:323] From <ipython-input-1-ddcb88721579>:197: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


W0711 14:32:38.357663 140399233443584 deprecation.py:323] From /home/student/anaconda3/envs/CPU/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Start and end (0, 5)
idx:  0  Epoch:  2  loss:  270.91507  loss_sent:  2.7077875  loss_word:  257.37616  Time cost:  125.41162586212158
Start and end (5, 10)
idx:  5  Epoch:  2  loss:  260.9162  loss_sent:  2.1245575  loss_word:  250.2934  Time cost:  74.912433385849
Start and end (10, 15)
idx:  10  Epoch:  2  loss:  210.05959  loss_sent:  1.9769504  loss_word:  200.17484  Time cost:  9.07091999053955
Start and end (15, 20)
idx:  15  Epoch:  2  loss:  238.28853  loss_sent:  4.0089884  loss_word:  218.24356  Time cost:  7.139372110366821
Start and end (20, 25)
idx:  20  Epoch:  2  loss:  229.59947  loss_sent:  2.3953362  loss_word:  217.62276  Time cost:  7.055136680603027
Start and end (25, 30)
idx:  25  Epoch:  2  loss:  265.02457  loss_sent:  2.1991463  loss_word:  254.02881  Time cost:  6.837973356246948
Start and end (30, 35)
idx:  30  Epoch:  2  loss:  230.43161  loss_sent:  1.974234  loss_word:  220.56049  Time cost:  6.759197950363159
Start and end (35, 40)
idx:  35  Epoch:  2  

KeyboardInterrupt: 

In [1]:
#WITH SCOPE FOR SESSION
#!/usr/bin/env python
# coding=utf-8

__author__ = "Xinpeng.Chen"

import os
import sys
import time
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import random

import h5py
import ipdb

import tensorflow as tf


# ------------------------------------------------------------------------------------------------------
# Initialization class
#  1. Pooling the visual features into a single dense feature
#  2. Then, build sentence LSTM, word LSTM
# ------------------------------------------------------------------------------------------------------
class RegionPooling_HierarchicalRNN():
    def __init__(self, n_words,
                       batch_size,
                       num_boxes,
                       feats_dim,
                       project_dim,
                       sentRNN_lstm_dim,
                       sentRNN_FC_dim,
                       wordRNN_lstm_dim,
                       S_max,
                       N_max,
                       word_embed_dim,
                       bias_init_vector=None):

        self.n_words = n_words
        self.batch_size = batch_size
        self.num_boxes = num_boxes # 50
        self.feats_dim = feats_dim # 4096
        self.project_dim = project_dim # 1024
        self.S_max = S_max # 6
        self.N_max = N_max # 50
        self.word_embed_dim = word_embed_dim # 1024

        self.sentRNN_lstm_dim = sentRNN_lstm_dim # 512 hidden size
        self.sentRNN_FC_dim = sentRNN_FC_dim # 1024 in fully connected layer
        self.wordRNN_lstm_dim = wordRNN_lstm_dim # 512 hidden size
	


        # word embedding, parameters of embedding
        # embedding shape: n_words x wordRNN_lstm_dim
        with tf.device('/cpu:0'):
            self.Wemb = tf.Variable(tf.random_uniform([n_words, word_embed_dim], -0.1, 0.1), name='Wemb')
        print("Wemb output>>>>>>>>>>>>",self.Wemb,tf.shape(self.Wemb))
        #self.bemb = tf.Variable(tf.zeros([word_embed_dim]), name='bemb')

        # regionPooling_W shape: 4096 x 1024
        # regionPooling_b shape: 1024
        self.regionPooling_W = tf.Variable(tf.random_uniform([feats_dim, project_dim], -0.1, 0.1), name='regionPooling_W')
        self.regionPooling_b = tf.Variable(tf.zeros([project_dim]), name='regionPooling_b')

        # sentence LSTM
        self.sent_LSTM = tf.nn.rnn_cell.BasicLSTMCell(sentRNN_lstm_dim, state_is_tuple=True)

        # logistic classifier
        self.logistic_Theta_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, 2], -0.1, 0.1), name='logistic_Theta_W')
        self.logistic_Theta_b = tf.Variable(tf.zeros(2), name='logistic_Theta_b')

        # fc1_W: 512 x 1024, fc1_b: 1024
        # fc2_W: 1024 x 1024, fc2_b: 1024
        self.fc1_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, sentRNN_FC_dim], -0.1, 0.1), name='fc1_W')
        self.fc1_b = tf.Variable(tf.zeros(sentRNN_FC_dim), name='fc1_b')
        self.fc2_W = tf.Variable(tf.random_uniform([sentRNN_FC_dim, 1024], -0.1, 0.1), name='fc2_W')
        self.fc2_b = tf.Variable(tf.zeros(1024), name='fc2_b')
        def get_a_cell(lstm_size):
          lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
          return lstm
        # word LSTM
        self.word_LSTM = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)
        #self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([self.word_LSTM] * 2, state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([get_a_cell(wordRNN_lstm_dim) for i in range(2)], state_is_tuple=True)
        #self.word_LSTM2 = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)


        self.embed_word_W = tf.Variable(tf.random_uniform([wordRNN_lstm_dim, n_words], -0.1,0.1), name='embed_word_W')
        if bias_init_vector is not None:
            self.embed_word_b = tf.Variable(bias_init_vector.astype(np.float32), name='embed_word_b')
        else:
            self.embed_word_b = tf.Variable(tf.zeros([n_words]), name='embed_word_b')

    def build_model(self):
        # receive the feats in the current image
        # it's shape is 10 x 50 x 4096
        # tmp_feats: 500 x 4096
        feats = tf.placeholder(tf.float32, [self.batch_size, self.num_boxes, self.feats_dim])
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 500 x 4096 * 4096 x 1024 --> 500 x 1024
        # project_vec: 10 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [self.batch_size, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # receive the [continue:0, stop:1] lists
        # example: [0, 0, 0, 0, 1, 1], it means this paragraph has five sentences
        num_distribution = tf.placeholder(tf.int32, [self.batch_size, self.S_max])

        # receive the ground truth words, which has been changed to idx use word2idx function
        captions = tf.placeholder(tf.int32, [self.batch_size, self.S_max, self.N_max+1])
        #print("Captions:>>>>>>>>>>>>>>>>>>>>>>>",captions)
        captions_masks = tf.placeholder(tf.float32, [self.batch_size, self.S_max, self.N_max+1])

        # ---------------------------------------------------------------------------------------------------------------------
        # The method which initialize the state, is refered from below sites:
        # 1. http://stackoverflow.com/questions/38241410/tensorflow-remember-lstm-state-for-next-batch-stateful-lstm/38417699
        # 2. https://www.tensorflow.org/api_docs/python/rnn_cell/classes_storing_split_rnncell_state#LSTMStateTuple
        # 3. https://medium.com/@erikhallstrm/using-the-tensorflow-lstm-api-3-7-5f2b97ca6b73#.u4w9z6h0h
        # ---------------------------------------------------------------------------------------------------------------------
        sent_state = self.sent_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state = self.word_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state1 = self.word_LSTM1.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #word_state2 = self.word_LSTM2.zero_state(batch_size=self.batch_size, dtype=tf.float32)
        #sent_state = tf.zeros([self.batch_size, self.sent_LSTM1.state_size])
        #word_state1 = tf.zeros([self.batch_size, self.word_LSTM1.state_size])
        #word_state2 = tf.zeros([self.batch_size, self.word_LSTM2.state_size])

        probs = []
        loss = 0.0
        loss_sent = 0.0
        loss_word = 0.0
        lambda_sent = 5.0
        lambda_word = 1.0

        print('Start build model:')
        #----------------------------------------------------------------------------------------------
        # Hierarchical RNN: sentence RNN and words RNN
        # The word RNN has the max number, N_max = 50, the number in the papar is 50
        #----------------------------------------------------------------------------------------------
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            with tf.variable_scope('sent_LSTM',reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            # sent_state is a tuple, sent_state = (c, h)
            # 'c': shape=(1, 512) dtype=float32, 'h': shape=(1, 512) dtype=float32
            # The loss here, I refer from the web which is very helpful for me:
            # 1. http://stackoverflow.com/questions/34240703/difference-between-tensorflow-tf-nn-softmax-and-tf-nn-softmax-cross-entropy-with
            # 2. http://stackoverflow.com/questions/35277898/tensorflow-for-binary-classification
            # 3. http://stackoverflow.com/questions/35226198/is-this-one-hot-encoding-in-tensorflow-fast-or-flawed-for-any-reason
            # 4. http://stackoverflow.com/questions/35198528/reshape-y-train-for-binary-text-classification-in-tensorflow
            sentRNN_logistic_mu = tf.nn.xw_plus_b( sent_output, self.logistic_Theta_W, self.logistic_Theta_b )
            sentRNN_label = tf.stack([ 1 - num_distribution[:, i], num_distribution[:, i] ])
            sentRNN_label = tf.transpose(sentRNN_label)
            sentRNN_loss = tf.nn.softmax_cross_entropy_with_logits(logits=sentRNN_logistic_mu, labels=sentRNN_label)
            sentRNN_loss = tf.reduce_sum(sentRNN_loss)/self.batch_size
            loss += sentRNN_loss * lambda_sent
            loss_sent += sentRNN_loss

            # the begining input of word_LSTM is topic vector, and DON'T compute the loss
            # This is follow the paper: Show and Tell
            #word_state = self.word_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
            #with tf.variable_scope('word_LSTM'):
            #    word_output, word_state = self.word_LSTM(sent_topic_vec)
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, captions[:, i, j])

                with tf.variable_scope('word_LSTM',reuse=tf.AUTO_REUSE):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                # How to make one-hot encoder, I refer from this excellent web:
                # http://stackoverflow.com/questions/33681517/tensorflow-one-hot-encoder
                labels = tf.reshape(captions[:, i, j+1], [-1, 1])
                #print("Labels and its shape +++++++++++++++",labels,tf.shape(labels))
                indices = tf.reshape(tf.range(0, self.batch_size, 1), [-1, 1])
                #print("Indices and its shape +++++++++++++++",indices,tf.shape(indices))
                concated = tf.concat([indices, labels],1)
                #print("Concated+++++++++++++++++++",concated)
                print("Success")
                onehot_labels = tf.sparse_to_dense(concated, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)

                # At each timestep the hidden state of the last LSTM layer is used to predict a distribution
                # over the words in the vocbulary
                logit_words = tf.nn.xw_plus_b(word_output[:], self.embed_word_W, self.embed_word_b)
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=onehot_labels)
                cross_entropy = cross_entropy * captions_masks[:, i, j]
                loss_wordRNN = tf.reduce_sum(cross_entropy) / self.batch_size
                loss += loss_wordRNN * lambda_word
                loss_word += loss_wordRNN

        return feats, num_distribution, captions, captions_masks, loss, loss_sent, loss_word

    def generate_model(self):
        # feats: 1 x 50 x 4096
        feats = tf.placeholder(tf.float32, [1, self.num_boxes, self.feats_dim])
        # tmp_feats: 50 x 4096
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 50 x 4096 * 4096 x 1024 + 1024 --> 50 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [1, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # initialize the sent_LSTM state
        sent_state = self.sent_LSTM.zero_state(batch_size=1, dtype=tf.float32)

        # save the generated paragraph to list, here I named generated_sents
        generated_paragraph = []

        # pred
        pred_re = []

        # T_stop: run the sentence RNN forward until the stopping probability p_i (STOP) exceeds a threshold T_stop
        T_stop = tf.constant(0.5)

        # Start build the generation model
        print('Start build the generation model: ')

        # sentence RNN
        #word_state = self.word_LSTM.zero_state(batch_size=1, dtype=tf.float32)
        #with tf.variable_scope('word_LSTM'):
        #    word_output, word_state = self.word_LSTM(sent_topic_vec, word_state)
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            # sent_state:
            # LSTMStateTuple(c=<tf.Tensor 'sent_LSTM/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
            #                h=<tf.Tensor 'sent_LSTM/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>)
            with tf.variable_scope('sent_LSTM',reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            # self.fc1_W: 512 x 1024, self.fc1_b: 1024
            # hidden1: 1 x 1024
            # sent_topic_vec: 1 x 1024
            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            sentRNN_logistic_mu = tf.nn.xw_plus_b(sent_output, self.logistic_Theta_W, self.logistic_Theta_b)
            pred = tf.nn.softmax(sentRNN_logistic_mu)
            pred_re.append(pred)

            # save the generated sentence to list, named generated_sent
            generated_sent = []

            # initialize the word LSTM state
            #word_state = self.word_LSTM.zero_state(batch_size=1, dtype=tf.float32)
            #with tf.variable_scope('word_LSTM'):
            #    word_output, word_state = self.word_LSTM(sent_topic_vec, word_state)
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            # word RNN, unrolled to N_max time steps
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                if j == 0:
                    with tf.device('/cpu:0'):
                        # get word embedding of BOS (index = 0)
                        current_embed = tf.nn.embedding_lookup(self.Wemb, tf.zeros([1], dtype=tf.int64))

                with tf.variable_scope('word_LSTM',reuse=tf.AUTO_REUSE):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                # word_state:
                # (
                #     LSTMStateTuple(c=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell0/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
                #                    h=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell0/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>),
                #     LSTMStateTuple(c=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell1/BasicLSTMCell/add_2:0' shape=(1, 512) dtype=float32>,
                #                    h=<tf.Tensor 'word_LSTM_152/MultiRNNCell/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 512) dtype=float32>)
                # )
                logit_words = tf.nn.xw_plus_b(word_output, self.embed_word_W, self.embed_word_b)
                max_prob_index = tf.argmax(logit_words, 1)[0]
                generated_sent.append(max_prob_index)

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, max_prob_index)
                    current_embed = tf.expand_dims(current_embed, 0)

            generated_paragraph.append(generated_sent)

        return feats, generated_paragraph, pred_re, sent_topic_vec


# -----------------------------------------------------------------------------------------------------
# Preparing Functions
# -----------------------------------------------------------------------------------------------------
def preProBuildWordVocab(sentence_iterator, word_count_threshold=5):
    # borrowed this function from NeuralTalk
    print('preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, ))

    word_counts = {}
    nsents = 0

    for sent in sentence_iterator:
        nsents += 1
        tmp_sent = sent.lower().split(' ')
        if '' in tmp_sent:
            tmp_sent.remove('')

        for w in tmp_sent:
           word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print('filtered words from %d to %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '<bos>'
    ixtoword[1] = '<eos>'
    ixtoword[2] = '<pad>'
    ixtoword[3] = '<unk>'

    wordtoix = {}
    wordtoix['<bos>'] = 0
    wordtoix['<eos>'] = 1
    wordtoix['<pad>'] = 2
    wordtoix['<unk>'] = 3

    for idx, w in enumerate(vocab):
        wordtoix[w] = idx + 4
        ixtoword[idx+4] = w

    word_counts['<eos>'] = nsents
    word_counts['<bos>'] = nsents
    word_counts['<pad>'] = nsents
    word_counts['<unk>'] = nsents

    bias_init_vector = np.array([1.0 * word_counts[ ixtoword[i] ] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range

    return wordtoix, ixtoword, bias_init_vector


#######################################################################################################
# Parameters Setting
#######################################################################################################
batch_size = 5 # Being support batch_size
num_boxes = 50 # number of Detected regions in each image
feats_dim = 4096 # feature dimensions of each regions
project_dim = 1024 # project the features to one vector, which is 1024 dimensions

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words

S_max = 6
N_max = 50
T_stop = 0.5

n_epochs = 500
learning_rate = 0.0001


#######################################################################################################
# Word vocubulary and captions preprocessing stage
#######################################################################################################
img2paragraph = pickle.load(open('./img2paragraph', 'rb'))
all_sentences = []
for key, paragraph in img2paragraph.items():
    for each_sent in paragraph[1]:
        each_sent.replace(',', ' ,')
        all_sentences.append(each_sent)
word2idx, idx2word, bias_init_vector = preProBuildWordVocab(all_sentences, word_count_threshold=2)
np.save('./idx2word_batch', idx2word)

img2paragraph_modify = {}
for img_name, img_paragraph in img2paragraph.items():
    img_paragraph_1 = img_paragraph[1]

    # img_paragraph_1 is a list
    # it may contain the element: '' or ' ', like this:
    # [["a man is walking"], ["the dog is running"], [""], [" "]]
    # so, we should remove them ' ' and '' element
    if '' in img_paragraph_1:
        img_paragraph_1.remove('')
    if ' ' in paragraph[1]:
        img_paragraph_1.remove(' ')

    # the number sents in each paragraph
    # if the sents is bigger than S_max,
    # we force the number of sents to be S_max
    img_num_sents = len(img_paragraph_1)
    if img_num_sents > S_max:
        img_num_sents = S_max

    # if a paragraph has 4 sentences
    # then the img_num_distribution will be like this:
    # [0, 0, 0, 1, 1, 1]
    img_num_distribution = np.zeros([S_max], dtype=np.int32)
    img_num_distribution[img_num_sents-1:] = 1

    # we multiply the number 2, because the <pad> is encoded into 2
    img_captions_matrix = np.ones([S_max, N_max+1], dtype=np.int32) * 2 # zeros([6, 50])
    for idx, img_sent in enumerate(img_paragraph_1):
        # the number of sentences is img_num_sents
        if idx == img_num_sents:
            break

        # because we treat the ',' as a word
        img_sent = img_sent.replace(',', ' ,')

        # Because I have preprocess the paragraph_v1.json file in VScode before,
        # and I delete all the 2, 3, 4...bankspaces
        # so, actually, the 'elif' code will never run
        if img_sent[0] == ' ' and img_sent[1] != ' ':
            img_sent = img_sent[1:]
        elif img_sent[0] == ' ' and img_sent[1] == ' ' and img_sent[2] != ' ':
            img_sent = img_sent[2:]

        # Be careful the last part in a sentence, like this:
        # '...world.'
        # '...world. '
        if img_sent[-1] == '.':
            img_sent = img_sent[0:-1]
        elif img_sent[-1] == ' ' and img_sent[-2] == '.':
            img_sent = img_sent[0:-2]

        # Last, we add the <bos> and the <eos> in each sentences
        img_sent = '<bos> ' + img_sent + ' <eos>'

        # translate each word in a sentence into the unique number in word2idx dict
        # when we meet the word which is not in the word2idx dict, we use the mark: <unk>
        for idy, word in enumerate(img_sent.lower().split(' ')):
            # because the biggest number of words in a sentence is N_max, here is 50
            if idy == N_max:
                break

            if word in word2idx:
                img_captions_matrix[idx, idy] = word2idx[word]
            else:
                img_captions_matrix[idx, idy] = word2idx['<unk>']

    # Pay attention, the value type 'img_name' here is NUMBER, I change it to STRING type
    img2paragraph_modify[str(img_name)] = [img_num_distribution, img_captions_matrix]

with open('./img2paragraph_modify_batch', 'wb') as f:
    pickle.dump(img2paragraph_modify, f)


#######################################################################################################
# Train, validation and testing stage
#######################################################################################################
def train():
    ##############################################################################
    # some preparing work
    ##############################################################################
    model_path = './models_batch/'
    train_feats_path = './im2p_val_output.h5'
    train_output_file = h5py.File(train_feats_path, 'r')
    train_feats = train_output_file.get('feats')
    train_imgs_full_path_lists = open('./imgs_val_path.txt').read().splitlines()
    train_imgs_names = list(map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists))
    print("Train Images Names",train_imgs_names)

    # Model Initialization:
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    with tf.variable_scope(tf.get_variable_scope()) as scope:
        model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                              batch_size = batch_size,
                                              num_boxes = num_boxes,
                                              feats_dim = feats_dim,
                                              project_dim = project_dim,
                                              sentRNN_lstm_dim = sentRNN_lstm_dim,
                                              sentRNN_FC_dim = sentRNN_FC_dim,
                                              wordRNN_lstm_dim = wordRNN_lstm_dim,
                                              S_max = S_max,
                                              N_max = N_max,
                                              word_embed_dim = word_embed_dim,
                                              bias_init_vector = bias_init_vector)

        tf_feats, tf_num_distribution, tf_captions_matrix, tf_captions_masks, tf_loss, tf_loss_sent, tf_loss_word = model.build_model()
    #sess = tf.Session()
    with tf.Session() as sess:
        saver = tf.train.Saver(max_to_keep=500, write_version=1)
        with tf.variable_scope('optimizer',reuse= tf.AUTO_REUSE):
            train_op = tf.train.AdamOptimizer(learning_rate).minimize(tf_loss)
        tf.global_variables_initializer().run(session=sess)
        print("Done w session")

        # when you want to train the model from the front model
        #new_saver = tf.train.Saver(max_to_keep=500)
        #new_saver = tf.train.import_meta_graph('./models_batch/model-92.meta')
        #new_saver.restore(sess, tf.train.latest_checkpoint('./models_batch/'))

        all_vars = tf.trainable_variables()

        # open a loss file to record the loss value
        #loss_fd = open('loss_batch.txt', 'a')
        img2idx = {}
        for idx, img in enumerate(train_imgs_names):
            img2idx[img] = idx
        print("Img2idx",img2idx)

        # plt draw the loss curve
        # refer from: http://stackoverflow.com/questions/11874767/real-time-plotting-in-while-loop-with-matplotlib
        
        loss_to_draw = [315.90247,265.50665,243.4754,229.64827,219.55573,211.94145,204.5995,197.81664,192.10925,186.46211,180.70108,175.55612,170.3046,165.09123,160.21469,160.2537961546135,154.08637935852528,149.32272485827923,144.4875265452814,139.37248678689718,134.57183133995056,129.8052043039155,125.52713392114163,120.7494116071415,115.9398527387333,111.30544829552889,106.93300072288037,102.37911339288473,97.98938349818707,93.8994490746212,89.69688967054606,84.13689,79.04315,75.386566,71.634445,67.22706,63.297745,59.791893,58.797394,54.47476,50.78446,47.608204,44.824448,42.131664,39.22608,36.267944,33.43973,30.94212,29.161013,27.789131,25.733274,23.194603,21.0445,19.291529,18.102518,17.157877,17.008978,14.93577,13.9712105,12.671065,11.739764,10.853795,10.301606,9.915598,9.00906,8.191635,7.7371345,7.485827,7.1387715,7.2393036,7.0062246,6.03907,5.117557,4.7625046,4.913513,4.82957,5.0153527,4.8877826,4.4756083,3.8244674,7.6418276,7.2976747,6.7312045,6.5270514,6.6875095,5.9723153,4.704945,4.7736664,5.8504386,5.804864,4.7677197,3.7789633,3.6227777,3.7564394,4.2505994,4.3439636,3.6212943,3.0515492,3.0061777,3.1802363,3.4463785,3.1947653,2.5623198,3.1410942,3.497574,3.2585106,2.8631153,315.3198,264.98257,243.43536,230.01776,219.96605,211.50975]  #Have to hard code here once it crashes
        print("Before epoch loop->>>>>>>>>>>>>>>>>>>>>>>>>>>>> the len is",len(loss_to_draw))
        try:
            saver.restore(sess, './models_batch/model-112')
            print("pretrained model loaded successfully")
        except:
            print("fail to load pretrained model")
            pass
        
        
        for epoch in range(113, n_epochs):
            loss_to_draw_epoch = []
            # disorganize the order
            random.shuffle(train_imgs_names)

            for start, end in zip(range(0, len(train_imgs_names), batch_size),
                                  range(batch_size, len(train_imgs_names), batch_size)):
                loss_fd = open('loss_batch.txt', 'a')

                start_time = time.time()
                print("Start and end",(start,end))
                img_name = train_imgs_names[start:end]
                current_feats_index = map(lambda x: img2idx[x], img_name)
                current_feats = np.asarray( list(map(lambda x: train_feats[x], current_feats_index) ))

                current_num_distribution = np.asarray( list(map(lambda x: img2paragraph_modify[x][0], img_name) ))
                current_captions_matrix = np.asarray( list(map(lambda x: img2paragraph_modify[x][1], img_name) ))

                current_captions_masks = np.zeros( (current_captions_matrix.shape[0], current_captions_matrix.shape[1], current_captions_matrix.shape[2]) )
                # find the non-zero element
                nonzeros = np.array( list(map(lambda each_matrix: np.array( list(map(lambda x: (x != 2).sum() + 1, each_matrix ) )), current_captions_matrix ) ))
                for i in range(batch_size):
                    for ind, row in enumerate(current_captions_masks[i]):
                        row[:(nonzeros[i, ind]-1)] = 1

                # shape of current_feats: batch_size x 50 x 4096
                # shape of current_num_distribution: batch_size x 6
                # shape of current_captions_matrix: batch_size x 6 x 50
                _, loss_val, loss_sent, loss_word= sess.run(
                                    [train_op, tf_loss, tf_loss_sent, tf_loss_word],
                                    feed_dict={
                                               tf_feats: current_feats,
                                               tf_num_distribution: current_num_distribution,
                                               tf_captions_matrix: current_captions_matrix,
                                               tf_captions_masks: current_captions_masks
                                    })

                # append loss to list in a epoch
                loss_to_draw_epoch.append(loss_val)

                # running information
                print('idx: ', start, ' Epoch: ', epoch, ' loss: ', loss_val, ' loss_sent: ', loss_sent, ' loss_word: ', loss_word, ' Time cost: ', str((time.time() - start_time)))
                loss_fd.write('start: '+ str(start) +' end: ' + str(end) +' epoch ' + str(epoch) + ' loss ' + str(loss_val) + '\n')
                loss_fd.close()
            # draw loss curve every epoch
            loss_to_draw.append(np.mean(loss_to_draw_epoch)) #This is what weve to write in a temp file once it is about to crash
            plt_save_dir = './loss_imgs'
            plt_save_img_name = str(epoch) + '.png'
            plt.plot(range(len(loss_to_draw)), loss_to_draw, color='g')
            plt.grid(True)
            plt.savefig(os.path.join(plt_save_dir, plt_save_img_name))
            MyFile=open('temporary.txt','a')
            if np.mod(epoch, 1) == 0:
                #the list loss_to_draw needs to be saved in a temp file
                l=loss_to_draw
                MyList=map(lambda x:str(x)+',', l)
                MyFile.writelines(MyList)
                MyFile.write('\t'+'<><><><><><><><><><><><><>'+'\n')
                MyFile.close()
                print("Successfully Written to temporary")
                print("Epoch ", epoch, " is done. Saving the model ...")
                saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
        #loss_fd.close()
315.90247,265.50665,243.4754,229.64827,219.55573,211.94145,204.5995,197.81664,192.10925,186.46211,180.70108,175.55612,170.3046,165.09123,160.21469,160.2537961546135,154.08637935852528,149.32272485827923,144.4875265452814,139.37248678689718,134.57183133995056,129.8052043039155,125.52713392114163,120.7494116071415,115.9398527387333,111.30544829552889,106.93300072288037,102.37911339288473,97.98938349818707,93.8994490746212,89.69688967054606,84.13689,79.04315,75.386566,71.634445,67.22706,63.297745,59.791893,58.797394,54.47476,50.78446,47.608204,44.824448,42.131664,39.22608,36.267944,33.43973,30.94212,29.161013,27.789131,25.733274,23.194603,21.0445,19.291529,18.102518,17.157877,17.008978,14.93577,13.9712105,12.671065,11.739764,10.853795,10.301606,9.915598,9.00906,8.191635,7.7371345,7.485827,7.1387715,7.2393036,7.0062246,6.03907,5.117557,4.7625046,4.913513,4.82957,5.0153527,4.8877826,4.4756083,3.8244674,7.6418276,7.2976747,6.7312045,6.5270514,6.6875095,5.9723153,4.704945,4.7736664,5.8504386,5.804864,4.7677197,3.7789633,3.6227777,3.7564394,4.2505994,4.3439636,3.6212943,3.0515492,3.0061777,3.1802363,3.4463785,3.1947653,2.5623198,3.1410942,3.497574,3.2585106,2.8631153

def test():
    
    start_time = time.time()
    # change the model path according to your environment
    model_path = './model-250'

    # It's very important to use Pandas to Series this idx2word dict
    # After this operation, we can use list to extract the word at the same time
    idx2word = pd.Series(np.load('./data/idx2word_batch.npy').tolist())

    test_feats_path = './data/im2p_test_output.h5'
    test_output_file = h5py.File(test_feats_path, 'r')
    test_feats = test_output_file.get('feats')

    test_imgs_full_path_lists = open('./densecap/imgs_test_path.txt').read().splitlines()
    test_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], test_imgs_full_path_lists)
    
    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    test_model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                               batch_size = batch_size,
                                               num_boxes = num_boxes,
                                               feats_dim = feats_dim,
                                               project_dim = project_dim,
                                               sentRNN_lstm_dim = sentRNN_lstm_dim,
                                               sentRNN_FC_dim = sentRNN_FC_dim,
                                               wordRNN_lstm_dim = wordRNN_lstm_dim,
                                               S_max = S_max,
                                               N_max = N_max,
                                               word_embed_dim = word_embed_dim,
                                               bias_init_vector = bias_init_vector)
    

    tf_feats, tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors = test_model.generate_model()
    sess = tf.InteractiveSession()
    #print("Before there >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n\n")

    saver = tf.train.Saver()
    saver.restore(sess, model_path)

    img2idx = {}
    for idx, img in enumerate(test_imgs_names):
        img2idx[img] = idx
    print("IM2idx:::::::::::",img2idx)

    test_fd = open('HRNN_results.txt', 'w')
    for idx, img_name in enumerate(test_imgs_names):
        print(idx, img_name)
        test_fd.write(img_name + '\n')

        each_paragraph = []
        current_paragraph = ""

        current_feats_index = img2idx[img_name]
        current_feats = test_feats[current_feats_index]
        current_feats = np.reshape(current_feats, [1, 50, 4096])

        generated_paragraph_indexes, pred, sent_topic_vectors = sess.run(
                                                                         [tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors],
                                                                         feed_dict={
                                                                             tf_feats: current_feats
                                                                         })

        #generated_paragraph = idx2word[generated_paragraph_indexes]
        for sent_index in generated_paragraph_indexes:
            each_sent = []
            for word_index in sent_index:
                each_sent.append(idx2word[word_index])
            each_paragraph.append(each_sent)

        for idx, each_sent in enumerate(each_paragraph):
            # if the current sentence is the end sentence of the paragraph
            # According to the probability distribution:
            # CONTINUE: [1, 0]
            # STOP    : [0, 1]
            # So, if the first item of pred is less than the T_stop
            # the generation process is break
            if pred[idx][0][0] <= T_stop:
                break
            current_sent = ''
            for each_word in each_sent:
                current_sent += each_word + ' '
            current_sent = current_sent.replace('<eos> ', '')
            current_sent = current_sent.replace('<pad> ', '')
            current_sent = current_sent + '.'
            current_sent = current_sent.replace(' .', '.')
            current_sent = current_sent.replace(' ,', ',')
            current_paragraph +=current_sent
            if idx != len(each_paragraph) - 1:
                current_paragraph += ' '

        test_fd.write(current_paragraph + '\n')
    test_fd.close()
    print("Time cost: " + str(time.time()-start_time))

train()


preprocessing word counts and creating vocab based on word count threshold 2
filtered words from 18418 to 9900
Train Images Names ['2336489', '2386636', '2317584', '2349895', '2348349', '2393268', '2338512', '2319338', '2318897', '2382861', '2359450', '2393810', '2379103', '2400913', '2381116', '2379607', '2372836', '1592156', '2388737', '2316733', '2328394', '2391668', '2341747', '2319741', '2355380', '2353667', '2316710', '2393529', '1160254', '1159529', '2392926', '2399813', '2335023', '2400892', '2367765', '2375715', '2395072', '2349745', '2384897', '2406617', '2373936', '2318431', '2410717', '2353681', '2414772', '2346311', '2369025', '2367716', '2342700', '2355768', '2361773', '2352565', '2356407', '2393837', '2351900', '2378739', '2394493', '2334825', '2346609', '2355839', '2406971', '2327224', '2407039', '2377971', '2367554', '2368478', '2327958', '2366303', '2360963', '2414977', '2416656', '2378809', '2407520', '2366857', '2374646', '2327849', '2376953', '2373691', '1143', '23

W0724 09:10:24.178529 140197742454528 deprecation.py:323] From <ipython-input-1-0434d5425a35>:69: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0724 09:10:24.230486 140197742454528 deprecation.py:323] From <ipython-input-1-0434d5425a35>:87: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


Wemb output>>>>>>>>>>>> <tf.Variable 'Wemb:0' shape=(9904, 1024) dtype=float32_ref> Tensor("Shape:0", shape=(2,), dtype=int32)


W0724 09:10:24.759396 140197742454528 deprecation.py:506] From /home/student/anaconda3/envs/CPU/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0724 09:10:24.765649 140197742454528 deprecation.py:506] From /home/student/anaconda3/envs/CPU/lib/python3.7/site-packages/tensorflow/python/ops/rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Start build model:


...

W0724 12:54:25.701179 140197742454528 saver.py:1134] *******************************************************
W0724 12:54:25.701931 140197742454528 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W0724 12:54:25.703800 140197742454528 saver.py:1136] Consider switching to the more efficient V2 format:
W0724 12:54:25.704243 140197742454528 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W0724 12:54:25.705804 140197742454528 saver.py:1138] now on by default.
W0724 12:54:25.706351 140197742454528 saver.py:1139] *******************************************************


Successfully Written to temporary
Epoch  116  is done. Saving the model ...
Start and end (0, 5)
idx:  0  Epoch:  117  loss:  306.0431  loss_sent:  2.2035294  loss_word:  295.02545  Time cost:  6.699774265289307
Start and end (5, 10)
idx:  5  Epoch:  117  loss:  195.7753  loss_sent:  2.1817558  loss_word:  184.86652  Time cost:  6.577568769454956
Start and end (10, 15)
idx:  10  Epoch:  117  loss:  254.49966  loss_sent:  1.750996  loss_word:  245.74469  Time cost:  6.66348123550415
Start and end (15, 20)
idx:  15  Epoch:  117  loss:  252.05351  loss_sent:  1.5523162  loss_word:  244.29192  Time cost:  6.678164482116699
Start and end (20, 25)
idx:  20  Epoch:  117  loss:  270.15604  loss_sent:  1.5204961  loss_word:  262.55353  Time cost:  6.657594442367554
Start and end (25, 30)
idx:  25  Epoch:  117  loss:  266.9403  loss_sent:  1.2484947  loss_word:  260.69788  Time cost:  6.559634447097778
Start and end (30, 35)
idx:  30  Epoch:  117  loss:  144.37234  loss_sent:  0.9423062  loss_wo

idx:  290  Epoch:  117  loss:  141.43349  loss_sent:  1.2407072  loss_word:  135.22993  Time cost:  6.52098274230957
Start and end (295, 300)
idx:  295  Epoch:  117  loss:  209.18544  loss_sent:  1.1002694  loss_word:  203.68408  Time cost:  6.543124437332153
Start and end (300, 305)
idx:  300  Epoch:  117  loss:  143.52861  loss_sent:  0.9501236  loss_word:  138.77802  Time cost:  6.546809196472168
Start and end (305, 310)
idx:  305  Epoch:  117  loss:  183.43944  loss_sent:  1.4701047  loss_word:  176.08891  Time cost:  6.514309883117676
Start and end (310, 315)
idx:  310  Epoch:  117  loss:  207.7017  loss_sent:  1.2908405  loss_word:  201.24748  Time cost:  6.60737419128418
Start and end (315, 320)
idx:  315  Epoch:  117  loss:  104.48891  loss_sent:  1.2189769  loss_word:  98.394035  Time cost:  6.553439140319824
Start and end (320, 325)
idx:  320  Epoch:  117  loss:  220.7632  loss_sent:  1.0452548  loss_word:  215.53693  Time cost:  6.708653450012207
Start and end (325, 330)
idx

idx:  580  Epoch:  117  loss:  284.6357  loss_sent:  1.0177848  loss_word:  279.54678  Time cost:  6.750156879425049
Start and end (585, 590)
idx:  585  Epoch:  117  loss:  271.42517  loss_sent:  1.4857883  loss_word:  263.99622  Time cost:  6.945807218551636
Start and end (590, 595)
idx:  590  Epoch:  117  loss:  215.48586  loss_sent:  1.1522871  loss_word:  209.7244  Time cost:  6.81252384185791
Start and end (595, 600)
idx:  595  Epoch:  117  loss:  214.94214  loss_sent:  1.2504609  loss_word:  208.68983  Time cost:  6.684616327285767
Start and end (600, 605)
idx:  600  Epoch:  117  loss:  252.13863  loss_sent:  1.2400454  loss_word:  245.9384  Time cost:  6.706771612167358
Start and end (605, 610)
idx:  605  Epoch:  117  loss:  172.0123  loss_sent:  1.0169923  loss_word:  166.92737  Time cost:  6.979783535003662
Start and end (610, 615)
idx:  610  Epoch:  117  loss:  285.20636  loss_sent:  1.1862113  loss_word:  279.27527  Time cost:  7.063607692718506
Start and end (615, 620)
idx:

KeyboardInterrupt: 